In [1]:
import tensorflow as tf
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
from nltk.tokenize.casual import TweetTokenizer
from w266_common import utils, vocabulary
import time
import datetime
import os


/home/saulgrimaldo1/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import csv
tokenizer = TweetTokenizer()
x_data = []
x_contexts = []
labels = []
sentences  = []
contexts = []
with open('merged_data_v3.csv', 'r') as csvfile:
    linereader = csv.reader(csvfile, delimiter = '|')
    for i, row in enumerate(linereader):
        if i == 0:
            continue
        sentence, context, sarcasm = row
        sentences.append(sentence)
        contexts.append(context)
        x_tokens = utils.canonicalize_words(tokenizer.tokenize(sentence))
        context_tokens = utils.canonicalize_words(tokenizer.tokenize(context))
        x_data.append(x_tokens)
        x_contexts.append(context_tokens)
        labels.append(int(sarcasm))


#rng = np.random.RandomState(5)
#rng.shuffle(x_data)  # in-place
#train_split_idx = int(0.7 * len(labels))
#test_split_idx  = int(0.9 * len(labels))

shuffle_indices = np.random.permutation(np.arange(len(labels)))
train_split_idx = int(0.7 * len(labels))
test_split_idx  = int(0.9 * len(labels))

train_indices = shuffle_indices[:train_split_idx]
validation_indices = shuffle_indices[train_split_idx:test_split_idx]
test_indices = shuffle_indices[test_split_idx:]


train_sentences = np.array(x_data)[train_indices]
train_contexts = np.array(x_contexts)[train_indices]
train_labels= np.array(labels)[train_indices] 
validation_sentences = np.array(x_data)[validation_indices]
validation_labels = np.array(labels)[validation_indices]
validation_contexts = train_contexts = np.array(x_contexts)[validation_indices]
test_sentences = np.array(x_data)[test_indices]  
test_contexts = train_contexts = np.array(x_contexts)[test_indices]
test_labels = np.array(labels)[test_indices]  

In [3]:
train_sentences.shape

(6108,)

In [4]:
def transform_labels(raw_label_set, size):
    label_set = []
    for label in raw_label_set:
        labels = [0] * size
        labels[label] = 1
        label_set.append(labels)
    return np.array(label_set)

expanded_train_labels = transform_labels(train_labels, 2)
expanded_validation_labels = transform_labels(validation_labels,2)
expanded_test_labels = transform_labels(test_labels,2)

In [5]:
a = [1,3,4,5,6,7,8,8,1,5,6,7]
a + ["<PADDING>"]*0

[1, 3, 4, 5, 6, 7, 8, 8, 1, 5, 6, 7]

In [6]:
class PaddingAndTruncating:
    def __init__(self, max_len):
        self.max_len = max_len
    
    def pad_or_truncate(self, sentence):
        sen_len = len(sentence)
        paddings = self.max_len - sen_len
        if paddings >=0:
            return sentence + ["<PADDING>"] * paddings
        return sentence[0:paddings]
        
PadAndTrunc = PaddingAndTruncating(10)
        
        
PadAndTrunc.pad_or_truncate(["the","angry","man","is","angry"])

['the',
 'angry',
 'man',
 'is',
 'angry',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>']

In [7]:
from tensorflow.contrib import learn

#max_len = max([len(sent) for sent  in train_sentences])
PadAndTrunc =PaddingAndTruncating(30)
train_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, train_sentences))
train_context_padded = list(map(PadAndTrunc.pad_or_truncate, train_contexts))
validation_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, validation_sentences))
validation_context_padded = list(map(PadAndTrunc.pad_or_truncate, validation_contexts))
test_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, test_sentences))
test_context_padded = list(map(PadAndTrunc.pad_or_truncate, test_contexts))

vocabulary_size = 2500
vocab = vocabulary.Vocabulary(utils.flatten(list(train_sentences_padded) + list(train_context_padded)), vocabulary_size)
train_s = np.array(list(map(vocab.words_to_ids, train_sentences_padded)))
train_c = np.array(list(map(vocab.words_to_ids, train_context_padded)))
validation_s = np.array(list(map(vocab.words_to_ids, validation_sentences_padded)))
validation_c = np.array(list(map(vocab.words_to_ids, validation_context_padded)))
test_s = np.array(list(map(vocab.words_to_ids, test_sentences_padded)))
test_c = np.array(list(map(vocab.words_to_ids, test_context_padded)))
train_s

array([[ 10,   5,   7, ...,  15,   3,   3],
       [  5,  85,   2, ...,  12,   3,   3],
       [ 85, 213,  30, ...,   4,  14,   3],
       ..., 
       [ 10,   5,   7, ...,   3,   3,   3],
       [739,  73,  96, ...,   3,   3,   3],
       [ 10,   5,   7, ...,   3,   3,   3]])

In [8]:
i = 3
("conv-maxpool-%s" % i)

'conv-maxpool-3'

In [9]:
class TextCNN(object):

    def __init__(
      self, sequence_length, num_classes, vocab_size, 
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x1 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x1")
        self.input_x2 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x2")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars1 = tf.nn.embedding_lookup(self.W, self.input_x1)
            self.embedded_chars_expanded1 = tf.expand_dims(self.embedded_chars1, -1)
            self.embedded_chars2 = tf.nn.embedding_lookup(self.W, self.input_x2)
            self.embedded_chars_expanded2 = tf.expand_dims(self.embedded_chars2, -1)

        # Create a convolution + avgpool layer for each filter size
        pooled_outputs = []

        for i, filter_size in enumerate(filter_sizes):
            with tf.variable_scope("conv-avgpool-%s" % i) as scope:
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]

                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
              
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded1,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h1 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh1")
               
                # Maxpooling over the outputs
                pooled = tf.nn.avg_pool(
                    h1,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                
                pooled_outputs.append(pooled)
                #pooled_outputs.append(pooled)
                scope.reuse_variables()
                 
                
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded2,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h2 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh2")
                # Maxpooling over the outputs
                pooled2 = tf.nn.avg_pool(
                    h2,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled2)

        # Combine all the pooled features

        num_filters_total = num_filters * len(filter_sizes) * 2

        self.h_pool = tf.concat(pooled_outputs, 1)

        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])


        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
        
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
           
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            self.labels = tf.argmax(self.input_y, 1)
            TP = tf.count_nonzero(self.predictions * self.labels)
            TN = tf.count_nonzero((self.predictions - 1) * (self.labels - 1))
            FP = tf.count_nonzero(self.predictions * (self.labels - 1))
            FN = tf.count_nonzero((self.predictions - 1) * self.labels)
            correct_predictions = tf.equal(self.predictions, self.labels)
            self.precision = TP / (TP + FP)
            self.recall = TP / (TP + FN)
            self.f1_score = 2 * self.precision * self.recall / (self.precision + self.recall)
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [10]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [11]:
learning_rate = 0.1

tf.flags.DEFINE_float("dev_sample_percentage", 0.75, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters

# embedding of 60 is best so far
tf.flags.DEFINE_integer("embedding_dim",50, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 50, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.4, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0, "L2 regularization lambda (default: 0.0)")

# Training parameters
batch_size = 100
tf.flags.DEFINE_integer("batch_size", batch_size, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 20, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================


with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():

        cnn = TextCNN(
            sequence_length=train_s.shape[1],
            num_classes=2,
            vocab_size=vocabulary_size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)
        
         # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdagradOptimizer(learning_rate)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        #vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch1, x_batch2, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch1, x_batch2, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            #r 
            step, summaries, loss, accuracy, recall, precision, f1 = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy,cnn.recall, cnn.precision, cnn.f1_score],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}, rec {:g}, pre {:g}, f1 {:g}".format(time_str, step, loss, accuracy, recall, precision, f1))# recall, precision, f1))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(train_s, train_c,  expanded_train_labels)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch1, x_batch2, y_batch = zip(*batch)
            train_step(x_batch1, x_batch1, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(validation_s, validation_c, expanded_validation_labels, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))
print("\nTest Set:")
dev_step(test_s, test_c, expanded_test_labels, writer=dev_summary_writer)
print("")



Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=100
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.75
DROPOUT_KEEP_PROB=0.4
EMBEDDING_DIM=50
EVALUATE_EVERY=20
FILTER_SIZES=1
L2_REG_LAMBDA=0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.neg
NUM_CHECKPOINTS=5
NUM_EPOCHS=100
NUM_FILTERS=50
POSITIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.pos

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/hist is illegal; using conv-avgpool-0/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/sparsity is illegal; using conv-avgpool-0/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/grad/hist is illegal; using conv-avgpool-0/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/grad

2018-04-17T05:44:31.838826: step 103, loss 0.660751, acc 0.68
2018-04-17T05:44:31.859883: step 104, loss 0.529219, acc 0.78
2018-04-17T05:44:31.880754: step 105, loss 0.542582, acc 0.81
2018-04-17T05:44:31.901431: step 106, loss 0.543616, acc 0.7
2018-04-17T05:44:31.922884: step 107, loss 0.524705, acc 0.8
2018-04-17T05:44:31.940227: step 108, loss 0.448085, acc 0.863014
2018-04-17T05:44:31.962247: step 109, loss 0.601412, acc 0.77
2018-04-17T05:44:31.983507: step 110, loss 0.495531, acc 0.78
2018-04-17T05:44:32.004189: step 111, loss 0.486158, acc 0.8
2018-04-17T05:44:32.024996: step 112, loss 0.492305, acc 0.75
2018-04-17T05:44:32.050256: step 113, loss 0.506012, acc 0.85
2018-04-17T05:44:32.071245: step 114, loss 0.534678, acc 0.73
2018-04-17T05:44:32.091938: step 115, loss 0.549939, acc 0.73
2018-04-17T05:44:32.112630: step 116, loss 0.497657, acc 0.73
2018-04-17T05:44:32.129140: step 117, loss 0.558126, acc 0.767123
2018-04-17T05:44:32.149716: step 118, loss 0.536376, acc 0.73
201

2018-04-17T05:44:35.151870: step 231, loss 0.505998, acc 0.76
2018-04-17T05:44:35.172433: step 232, loss 0.433505, acc 0.87
2018-04-17T05:44:35.192991: step 233, loss 0.477783, acc 0.8
2018-04-17T05:44:35.209687: step 234, loss 0.472208, acc 0.794521
2018-04-17T05:44:35.231043: step 235, loss 0.535786, acc 0.72
2018-04-17T05:44:35.252677: step 236, loss 0.688999, acc 0.62
2018-04-17T05:44:35.273453: step 237, loss 0.653346, acc 0.7
2018-04-17T05:44:35.295432: step 238, loss 0.447335, acc 0.84
2018-04-17T05:44:35.319200: step 239, loss 0.351795, acc 0.93
2018-04-17T05:44:35.339548: step 240, loss 0.393656, acc 0.84

Evaluation:
2018-04-17T05:44:35.437348: step 240, loss 0.563817, acc 0.725501, rec 0.876471, pre 0.665773, f1 0.756729

2018-04-17T05:44:35.459164: step 241, loss 0.409059, acc 0.88
2018-04-17T05:44:35.479815: step 242, loss 0.406653, acc 0.81
2018-04-17T05:44:35.496495: step 243, loss 0.433361, acc 0.821918
2018-04-17T05:44:35.517349: step 244, loss 0.449673, acc 0.79
2018-

2018-04-17T05:44:38.500018: step 356, loss 0.409817, acc 0.79
2018-04-17T05:44:38.520675: step 357, loss 0.429948, acc 0.84
2018-04-17T05:44:38.542861: step 358, loss 0.334968, acc 0.88
2018-04-17T05:44:38.563408: step 359, loss 0.313028, acc 0.93
2018-04-17T05:44:38.579899: step 360, loss 0.367856, acc 0.863014

Evaluation:
2018-04-17T05:44:38.677417: step 360, loss 0.581095, acc 0.660745, rec 0.943529, pre 0.59584, f1 0.730419

2018-04-17T05:44:38.704376: step 361, loss 0.356951, acc 0.85
2018-04-17T05:44:38.729904: step 362, loss 0.344846, acc 0.88
2018-04-17T05:44:38.750893: step 363, loss 0.472204, acc 0.81
2018-04-17T05:44:38.771610: step 364, loss 0.358528, acc 0.82
2018-04-17T05:44:38.793820: step 365, loss 0.430803, acc 0.79
2018-04-17T05:44:38.815055: step 366, loss 0.395318, acc 0.82
2018-04-17T05:44:38.838539: step 367, loss 0.309684, acc 0.87
2018-04-17T05:44:38.859961: step 368, loss 0.35482, acc 0.86
2018-04-17T05:44:38.884250: step 369, loss 0.298218, acc 0.890411
2018-

2018-04-17T05:44:42.115948: step 480, loss 0.809227, acc 0.529513, rec 0.989412, pre 0.508772, f1 0.671994

2018-04-17T05:44:42.137808: step 481, loss 0.345881, acc 0.84
2018-04-17T05:44:42.157957: step 482, loss 0.392612, acc 0.79
2018-04-17T05:44:42.178183: step 483, loss 0.295618, acc 0.88
2018-04-17T05:44:42.198497: step 484, loss 0.290149, acc 0.89
2018-04-17T05:44:42.218972: step 485, loss 0.31511, acc 0.86
2018-04-17T05:44:42.235273: step 486, loss 0.304912, acc 0.863014
2018-04-17T05:44:42.255619: step 487, loss 0.343377, acc 0.86
2018-04-17T05:44:42.276179: step 488, loss 0.353398, acc 0.84
2018-04-17T05:44:42.296602: step 489, loss 0.299698, acc 0.86
2018-04-17T05:44:42.323793: step 490, loss 0.339846, acc 0.87
2018-04-17T05:44:42.344373: step 491, loss 0.370657, acc 0.89
2018-04-17T05:44:42.365019: step 492, loss 0.325451, acc 0.88
2018-04-17T05:44:42.385991: step 493, loss 0.219223, acc 0.93
2018-04-17T05:44:42.410085: step 494, loss 0.323711, acc 0.86
2018-04-17T05:44:42.4

2018-04-17T05:44:45.150776: step 600, loss 0.516948, acc 0.79

Evaluation:
2018-04-17T05:44:45.246223: step 600, loss 0.4832, acc 0.768481, rec 0.589412, pre 0.901079, f1 0.71266

Saved model checkpoint to /home/saulgrimaldo1/sarcasm_detection_w266/runs/1523943868/checkpoints/model-600

2018-04-17T05:44:45.312177: step 601, loss 0.321097, acc 0.86
2018-04-17T05:44:45.333059: step 602, loss 0.34763, acc 0.81
2018-04-17T05:44:45.349563: step 603, loss 0.233603, acc 0.931507
2018-04-17T05:44:45.373684: step 604, loss 0.264111, acc 0.86
2018-04-17T05:44:45.395311: step 605, loss 0.313541, acc 0.86
2018-04-17T05:44:45.415952: step 606, loss 0.337978, acc 0.92
2018-04-17T05:44:45.436988: step 607, loss 0.346709, acc 0.87
2018-04-17T05:44:45.458107: step 608, loss 0.35488, acc 0.85
2018-04-17T05:44:45.478494: step 609, loss 0.319072, acc 0.84
2018-04-17T05:44:45.498655: step 610, loss 0.265014, acc 0.88
2018-04-17T05:44:45.518662: step 611, loss 0.260612, acc 0.91
2018-04-17T05:44:45.534872: 

2018-04-17T05:44:48.421945: step 720, loss 0.613746, acc 0.668768, rec 0.967059, pre 0.599125, f1 0.739874

2018-04-17T05:44:48.444447: step 721, loss 0.312428, acc 0.88
2018-04-17T05:44:48.465702: step 722, loss 0.221939, acc 0.91
2018-04-17T05:44:48.486232: step 723, loss 0.284197, acc 0.87
2018-04-17T05:44:48.506621: step 724, loss 0.288732, acc 0.89
2018-04-17T05:44:48.527083: step 725, loss 0.338742, acc 0.86
2018-04-17T05:44:48.548236: step 726, loss 0.21265, acc 0.9
2018-04-17T05:44:48.568633: step 727, loss 0.186796, acc 0.96
2018-04-17T05:44:48.589396: step 728, loss 0.409622, acc 0.8
2018-04-17T05:44:48.605885: step 729, loss 0.252968, acc 0.90411
2018-04-17T05:44:48.630421: step 730, loss 0.311581, acc 0.86
2018-04-17T05:44:48.650806: step 731, loss 0.232219, acc 0.9
2018-04-17T05:44:48.671136: step 732, loss 0.197253, acc 0.93
2018-04-17T05:44:48.692272: step 733, loss 0.280776, acc 0.88
2018-04-17T05:44:48.713409: step 734, loss 0.299755, acc 0.87
2018-04-17T05:44:48.73410

2018-04-17T05:44:51.432201: step 840, loss 0.26608, acc 0.89

Evaluation:
2018-04-17T05:44:51.528759: step 840, loss 0.9307, acc 0.558166, rec 0.977647, pre 0.524953, f1 0.683107

2018-04-17T05:44:51.552454: step 841, loss 0.246982, acc 0.9
2018-04-17T05:44:51.573716: step 842, loss 0.284419, acc 0.89
2018-04-17T05:44:51.595346: step 843, loss 0.285982, acc 0.85
2018-04-17T05:44:51.616248: step 844, loss 0.325203, acc 0.89
2018-04-17T05:44:51.640748: step 845, loss 0.198668, acc 0.92
2018-04-17T05:44:51.657472: step 846, loss 0.222954, acc 0.890411
2018-04-17T05:44:51.678569: step 847, loss 0.206, acc 0.92
2018-04-17T05:44:51.699682: step 848, loss 0.255468, acc 0.91
2018-04-17T05:44:51.720951: step 849, loss 0.210741, acc 0.9
2018-04-17T05:44:51.742211: step 850, loss 0.324878, acc 0.85
2018-04-17T05:44:51.763227: step 851, loss 0.349327, acc 0.83
2018-04-17T05:44:51.783676: step 852, loss 0.340765, acc 0.83
2018-04-17T05:44:51.804584: step 853, loss 0.195249, acc 0.92
2018-04-17T05:4